## Idea testing notebook for CA4023 Assignment 1

Here, I test out ideas for in the assignment.

The full movie reviews which I analysed for Part 3 of the assignment can be found in this notebook, in Section 3.2

### 1. Sentence Boundary Detection



In [78]:
import re
import io

In [79]:
f = io.open("part-one/example1.txt", mode="r", encoding="utf-8")
example = f.read().replace('\n', ' ')
f.close()

In [80]:
def add_newline(match_obj):
    return match_obj.group(0)+'\n'

In [81]:
example

"The difference for them can be significant. A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  To test the effectiveness of this sentence boundary detector, let's try some special cases!!! Ironic huh? That was a special case... So is this, and this too! Pi being 3.14159 is not really an edge case. Two more things: The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. How will we deal with those pesky edge cases??? Disclaimer: All characters are ficitonal. I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. I.e. liquid ibuprofen. J. K. Rowling wrote Harry Potter, but those initials did

In [86]:
# naiive sentence boundary detection
abbrv2 = 'Mr|Ms|Dr'
abbrv3 = 'Mrs|[Ee]\.[Gg]|[Ii]\.[Ee]|[Ee]tc'
abbrv4 = 'Prof|Miss'
initial='\s[A-Z]'

# check that previous word is not an abbreviation
lookbehind_abbrv='(?<!'+abbrv2+')(?<!'+abbrv3+')(?<!'+abbrv4+')' 

# check that previous word is not a single capital letter
lookbehind_initial='(?<!'+initial+')'

# check that sentence end is not followed by a lowercase letter
lookahead='(?![a-z])'

sentence_end_punct='[?!:]+\s+'

# this pattern is separate as initials are only followed by a period (J. K. Rowling, not J: K: Rowling)
sentence_end_period='[\.]+\s+'

end_re1 = lookbehind_abbrv + sentence_end_punct + lookahead  # main pattern match
end_re2 = lookbehind_initial + lookbehind_abbrv + sentence_end_period + lookahead # for catching initials
print(re.sub(end_re1+"|"+end_re2, add_newline, example))

The difference for them can be significant. 
A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. 
With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  
To test the effectiveness of this sentence boundary detector, let's try some special cases!!! 
Ironic huh? 
That was a special case... 
So is this, and this too! 
Pi being 3.14159 is not really an edge case. 
Two more things: 
The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! 
And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. 
How will we deal with those pesky edge cases??? 
Disclaimer: 
All characters are ficitonal. 
I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. 
I.e. liquid ibuprofen. 
J. K. Rowling wrote Harry Potter, but tho

- Sentences can end with different forms of punctuation "[.?!:]" .
- Mr., Mrs., Ms., Miss., Dr., Prof., are not sentence endings, but rather honorifics.

To capture as many normal cases as possible, I will assume the following:
- Correct sentence endings are always used in the input text files. That is, a space must follow a sentence-end punctuation to be considered a sentence ending, and sentences do not start with lowercase letters.
-  A sentence does not end with a single capital letter. As a result of this assumption, the rare case in which it does end with a capital letter will not be detected.


In [85]:
def sentence_split(text):
    '''
        Given text as input, return each sentence on a separate line.

    '''
    # known abbreviations of different lengths
    abbrv2 = 'Mr|Ms|Dr'
    abbrv3 = 'Mrs|[Ee]\.[Gg]|[Ii]\.[Ee]|[Ee]tc'
    abbrv4 = 'Prof|Miss'

    initial='\s[A-Z]'

    # check that previous word is not an abbreviation
    lookbehind_abbrv='(?<!'+abbrv2+')(?<!'+abbrv3+')(?<!'+abbrv4+')' 

    # check that previous word is not a single capital letter (assumed to be an initial)
    lookbehind_initial='(?<!'+initial+')'

    # check that sentence end is not followed by a lowercase letter
    lookahead='(?![a-z])'

    # matches sentences ending in non-period punctuation
    sentence_end_punct='[?!:]+\s+'

    # separate sentence end pattern as initials are only followed by a period (J. K. Rowling, not J: K: Rowling)
    sentence_end_period='[\.]+\s+'

    end_re1 = lookbehind_abbrv + sentence_end_punct + lookahead  # punctuation pattern match
    end_re2 = lookbehind_initial + end_re1 # for catching initials

    return print(re.sub(end_re1+"|"+end_re2, add_newline, example))
    
sentence_split(example)

The difference for them can be significant. A 10-year loan of €50,000 from you at zero interest would involve monthly payments of €417, or €5,000 a year. With An Post’s market-leading rate, the friend would be paying €525.45 a month, or over €6,300 a year and over €13,000 interest over the life of the loan.  To test the effectiveness of this sentence boundary detector, let's try some special cases!!! 
Ironic huh? 
That was a special case... So is this, and this too! 
Pi being 3.14159 is not really an edge case. Two more things: 
The capital of U.s.a. is Washington DC, but that isn't the end of a sentence! 
And George W. Bush was the 43rd president of America, but that wasn't a sentence ending either. How will we deal with those pesky edge cases??? 
Disclaimer: 
All characters are ficitonal. I went to Dr. Larry because I had a sore head, and he said Prof. Phillip's and Mrs. Mcgeoghs books had an antidote for that. I.e. liquid ibuprofen. J. K. Rowling wrote Harry Potter, but those initia

### 2. Bigram Language Model

In [1]:
from collections import defaultdict
corpus = ['<s> b c </s>', '<s> c c </s>', '<s> c b </s>', '<s> b b </s>']

In [6]:
# initialise dictionaries for storing frequency. These are used for computing probabilities

def train(corpus):
    '''
    Given a training corpus in the form of a list of sentences, return word and bigram frequency dictionaries
    for computing probabilities
    '''     
    word_freq = defaultdict(int)
    bigram_freq = defaultdict(int)

    for sentence in corpus:
        tokens = sentence.split()
        
        # update word frequency dicitonary
        for token in tokens:
            word_freq[token] += 1
        
        # update bigram frequency dictionary
        if len(tokens) >= 2:
            i = 0
            while i < len(tokens)-1:
                bigram_freq[tokens[i]+' '+tokens[i+1]] += 1
                i += 1
    
    print("Returning 2 dicitonaries. word_freq [0] and bigram_freq [1].")
    return word_freq, bigram_freq

word_freq, bigram_freq = train(corpus)

Returning 2 dicitonaries. word_freq [0] and bigram_freq [1].


In [7]:
print(word_freq)
print(bigram_freq)

defaultdict(<class 'int'>, {'<s>': 4, 'b': 4, 'c': 4, '</s>': 4})
defaultdict(<class 'int'>, {'<s> b': 2, 'b c': 1, 'c </s>': 2, '<s> c': 2, 'c c': 1, 'c b': 1, 'b </s>': 2, 'b b': 1})


In [9]:
# computing probabilities
def compute_prob(sentence, word_freq=word_freq, bigram_freq=bigram_freq):
    ''' 
    Given bigram and word frequencies from a coprus, return the probability of the input sentence.

    In a bigram model, P(w1, w2, ..., wn) = P(w1)*P(w2|w1)*P(w3|w2)*...*P(wn|wn-1).
    Since the model uses word frequencies in the corpus to approximate probabilities:
    P(wn|wn-1) = (Count of [wn-1, wn] bigram in corpus) / Count of wn-1 in corpus.

    '''
    n_corpus_tokens = sum(word_freq.values())
    tokens = sentence.split()
    i = 0
    prob = 1
    while i < len(tokens):
        curr = tokens[i]
        if i == 0:  # edge case: start of sentence
            prob *= (word_freq[curr]) / n_corpus_tokens
        else:
            prev = tokens[i-1]
            prob *= (bigram_freq[prev+" "+curr])/(word_freq[prev])
        i += 1
    
    return prob
compute_prob("<s> c </s>")

0.0625

### 3. Naiive Bayes Sentiment Polarity Classifier

#### 3.1 Model build

In [2]:
# create dataset from the text files
import pandas as pd
import numpy as np
import os
import sys
from collections import defaultdict
import string
import math
seed=42

pos_path = '../data/txt_sentoken/pos'
neg_path = '../data/txt_sentoken/neg'
pos_df = pd.DataFrame()
neg_df = pd.DataFrame()

# parse the files in positive directory
for filename in os.listdir(pos_path):
    with open(os.path.join(pos_path, filename)) as f:
        review = f.read()
        label='positive'
        
        # assign to train and test set depending on the filename
        which_set = 'train' if int(filename[2]) < 9 else 'test'
        row = pd.DataFrame({'review':[review], 'label':[label], 'set':[which_set]})

    pos_df = pd.concat([pos_df, row], axis=0)
print("Done Positive")

# parse the files in negative directory
for filename in os.listdir(neg_path):
    with open(os.path.join(neg_path, filename)) as f:
        review = f.read()
        label='negative'
        
        # assign to train and test set depending on the filename
        which_set = 'train' if int(filename[2]) < 9 else 'test'
        row = pd.DataFrame({'review':[review], 'label':[label], 'set':[which_set]})

    neg_df = pd.concat([neg_df, row], axis=0)
print("Done Negative")

df = pd.concat([pos_df, neg_df], axis=0).reset_index(drop=True)
train_df = df.loc[df.set=='train', :].drop(columns=['set'])
test_df = df.loc[df.set=='test', :].drop(columns=['set'])

Done Positive
Done Negative


In [3]:
train_df.head()

,review,label
0,films adapted from comic books have had plenty...,positive
1,every now and then a movie comes along from a ...,positive
2,you've got mail works alot better than it dese...,positive
3,""" jaws "" is a rare film that grabs your atten...",positive
4,moviemaking is a lot like being the general ma...,positive


In [4]:

def run_nb_model(train_df=train_df, test_df=test_df, precision=8):
    '''
        Given a training and testing dataframe as input containing columns [review, label], return the label predictions of
        the test data. If the testing dataframe has a label column, use it to compute the accuracy of the classifier. Otherwise, 
        just return the predictions.

        Arguments:
            train_df (pd.DataFrame): data frame with training reviews and their "positive" or "negative" labels
            test_df (pd.DataFrame): data frame with testing reviews. The predicted labels are appended to test_df when this function is run.
            precision (int): decimal point precision with which to return float values.
        
        Returns:
            predicted labels for test_df, and optionally the accuracy as a float with the specified precision.
    '''
    def tokenize_review(review):
        ''' Helper function to return tokens from a review as a list. '''
        tokens = []
        for token in review.split():
            token = token.strip(string.punctuation+' ').lower()
            
            # ignore empty tokens
            if token != ' ' and token != '':
                tokens.append(token)
        
        return tokens
    
    def calc_accuracy(y_true, y_pred):
        ''' Helper function to compute model accuracy given true labels and predictions '''
        n_correct = 0
        i = 0
        while i < len(y_true):
            if y_true[i] == y_pred[i]:
                n_correct += 1
            i += 1
        return n_correct / len(y_true)

    # build word count dictionaries for positive and negative reviews.
    pos_freq = defaultdict(int)
    neg_freq = defaultdict(int)

    # parse through each review in training data, tokenize and add tokens to their respective dictionaries
    for index, row in train_df.iterrows():
        for token in tokenize_review(row.review):
            # add tokens to the word counts
            if row.label == 'positive':
                pos_freq[token] += 1
            else:
                neg_freq[token] += 1
    
    n_pos_tokens = sum(pos_freq.values())
    n_neg_tokens = sum(neg_freq.values())
    
    # number of unique words in all reviews
    n_vocab = len(set(pos_freq.keys()).union(set(pos_freq.keys())))
    
    # find probability of review from both positive and negative frequency distributions.
    predictions = []
    lp_pos = []
    lp_neg = []
    for index, row in test_df.iterrows():
        # in this case, prior is not useful because we have same number of positive and negative documents
        log_prob_pos = 0
        log_prob_neg = 0
        for token in tokenize_review(row.review):
            # ignores tokens that didn't appear in the training reviews, includes add-1 smoothing

            # if the word is in each review class, update probability
            if (pos_freq[token] > 0) or (neg_freq[token] > 0):
                log_prob_pos += math.log((pos_freq[token]+1)/(n_pos_tokens+n_vocab))
                log_prob_neg += math.log((neg_freq[token]+1)/(n_neg_tokens+n_vocab))
                

        # whichever probability is highest determines the predicted label.
        ans = 'positive' if log_prob_pos >= log_prob_neg else 'negative'
        predictions.append(ans)
        lp_pos.append(log_prob_pos)
        lp_neg.append(log_prob_neg)

    
    # add the predictions, and log probabilities to the test dataframe for analysis
    test_df['pred_label'] = predictions
    test_df['logp_positive'] = lp_pos
    test_df['logp_negative'] = lp_neg
    if 'label' in test_df.columns:
        accuracy = round(calc_accuracy(y_true=test_df.label.tolist(), y_pred=predictions), precision)

    # returning pos_freq and neg_freq for checking sentiment of individual words in the next section
    return predictions, accuracy, pos_freq, neg_freq

    
predictions, accuracy, pos_freq, neg_freq = run_nb_model(train_df, test_df, precision=8)

In [5]:
accuracy

0.83

#### 3.2 Analysing predictions from Naive Bayes model

In [6]:
correct_sample = test_df.loc[test_df.label == test_df.pred_label].sample(5, random_state=seed)
incorrect_sample = test_df.loc[test_df.label != test_df.pred_label].sample(5, random_state=seed)

In [7]:
correct_sample['log_p_diff'] = np.abs(correct_sample.logp_positive - correct_sample.logp_negative)
incorrect_sample['log_p_diff'] = np.abs(incorrect_sample.logp_positive - incorrect_sample.logp_negative)

In [8]:
print("Correct Sample:")
correct_sample.loc[:, ['review', 'label', 'pred_label', 'log_p_diff']].round()

Correct Sample:


,review,label,pred_label,log_p_diff
1924,a number of critics have decided that it's ope...,negative,negative,38.0
1965,""" book "" should have remained in shadows \nbo...",negative,negative,24.0
998,steven spielberg's second epic film on world w...,positive,positive,65.0
968,insane ( but inspired ) musical about alferd p...,positive,positive,31.0
1921,note : some may consider portions of the follo...,negative,negative,58.0


In [9]:
print("Inorrect Samples:")
incorrect_sample.loc[:, ['review', 'label', 'pred_label', 'log_p_diff']].round()

Inorrect Samples:


,review,label,pred_label,log_p_diff
987,trees lounge is the directoral debut from one ...,positive,negative,0.0
994,"a thriller set in modern day seattle , that ma...",positive,negative,22.0
1954,"a new entry in the "" revisionist history "" gen...",negative,positive,6.0
1941,"set in harlem during the great depression , ri...",negative,positive,21.0
932,"armageddon , in itself , symbolizes everything...",positive,negative,56.0


In [17]:
incorrect_sample.loc[:, ['review', 'label', 'pred_label', 'log_p_diff']]

,review,label,pred_label,log_p_diff
987,trees lounge is the directoral debut from one ...,positive,negative,0.082181
994,"a thriller set in modern day seattle , that ma...",positive,negative,22.075108
1954,"a new entry in the "" revisionist history "" gen...",negative,positive,6.266019
1941,"set in harlem during the great depression , ri...",negative,positive,21.309421
932,"armageddon , in itself , symbolizes everything...",positive,negative,56.397478


In [10]:
# Full reviews which are analysed in ANALYSIS.md
# output may be too long to view here, should be optional to view in a text editor
print("CORRECT PREDICTIONS\n"+"-"*100)
for index, row in correct_sample.iterrows():
    print("label:", row.label, "\npredicted:", row.pred_label)
    print("Review:\n"+row.review+"\n")
    print("-"*100+"\n")

print("\nINCORRECT PREDICTIONS\n"+"-"*100)
# Full reviews which are analysed in ANALYSIS.md
# output may be too long to view here, should be optional to view in a text editor
for index, row in incorrect_sample.iterrows():
    print("label:", row.label, "\npredicted:", row.pred_label)
    print("Review:\n"+row.review+"\n")
    print("-"*100+"\n")


CORRECT PREDICTIONS
----------------------------------------------------------------------------------------------------
label: negative 
predicted: negative
Review:
a number of critics have decided that it's open season on freddie prize jr . , slamming the young actor as an utterly talentless pretty boy on career cruise control in sound-alike , disposable teen fluff like " head over heels , " " boys and girls , " " down to you " and " she's all that . " 
while the prinze oeuvre is hard to defend , his talent is not . 
i first saw him in the independent dark comedy " the house of yes , " where he gave a subtle , impressive performance as the younger brother in one of america's freakiest families . 
prinze has the acting chops ; he just needs to take a few supporting roles in some grown-up movies to show the non-believers that he has what it takes . 
 " summer catch " certainly won't help his case . 
inoffensive , but utterly generic , the baseball-related romantic comedy does little mo

In [11]:
def check_prob(word, pos_freq=pos_freq, neg_freq=neg_freq, precision=10):
    ''' Function to check the Laplace-smoothed probability that a word occurs in a positive or negative review.
        Uses word frequency dictionaries to compute the probabilities.

        Returns:
        probability of word occuring given review is positive[0]/negative[1]
    '''
    n_pos_tokens = sum(pos_freq.values())
    n_neg_tokens = sum(neg_freq.values())
    n_vocab = len(set(pos_freq.keys()).union(set(neg_freq.keys())))

    prob_pos = round((pos_freq[word]+1)/(n_pos_tokens+n_vocab), precision)
    prob_neg = round((neg_freq[word]+1)/(n_neg_tokens+n_vocab), precision)

    return prob_pos, prob_neg

In [12]:
# Example 1 candidate words
candidates = ['talent', 'love', 'disposable', 'utterly']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    

talent
Probability in positive review: 0.000119291
Probability in negative review: 0.0002028139

love
Probability in positive review: 0.0008803375
Probability in negative review: 0.0006637546

disposable
Probability in positive review: 7.5501e-06
Probability in negative review: 1.1733e-05

utterly
Probability in positive review: 5.13404e-05
Probability in negative review: 6.36936e-05



In [13]:
# Example 2 candidate words
candidates = ['stupidly', 'predictable', 'poor', 'ruined']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    
    

stupidly
Probability in positive review: 1.51e-06
Probability in negative review: 1.00569e-05

predictable
Probability in positive review: 8.30507e-05
Probability in negative review: 0.0001843763

poor
Probability in positive review: 9.36208e-05
Probability in negative review: 0.0002279561

ruined
Probability in positive review: 2.11402e-05
Probability in negative review: 3.01707e-05



In [14]:
# Example 3 candidate words
candidates = ['masterpiece', 'powerful', 'astonishing', 'miracle']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))    
    

masterpiece
Probability in positive review: 9.51308e-05
Probability in negative review: 3.51991e-05

powerful
Probability in positive review: 0.0001812015
Probability in negative review: 7.54267e-05

astonishing
Probability in positive review: 2.71802e-05
Probability in negative review: 1.50853e-05

miracle
Probability in positive review: 1.20801e-05
Probability in negative review: 6.7046e-06



In [15]:
# Example 4 candidate words
candidates = ['beautiful', 'entertaining', 'achievement', 'best']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))   

beautiful
Probability in positive review: 0.0002687823
Probability in negative review: 0.0001458249

entertaining
Probability in positive review: 0.0002446221
Probability in negative review: 0.000192757

achievement
Probability in positive review: 3.62403e-05
Probability in negative review: 2.84945e-05

best
Probability in positive review: 0.0011068395
Probability in negative review: 0.0007509143



In [16]:
# Example 5 candidate words
candidates = ['joyless', 'lacking', 'uninvolving', 'ridiculous']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))   

joyless
Probability in positive review: 1.51e-06
Probability in negative review: 5.0284e-06

lacking
Probability in positive review: 3.77503e-05
Probability in negative review: 5.86652e-05

uninvolving
Probability in positive review: 4.53e-06
Probability in negative review: 3.3523e-05

ridiculous
Probability in positive review: 3.17103e-05
Probability in negative review: 0.0001726433



In [18]:
# Example 6 candidate words
candidates = ['awfully', 'slow', 'pestering', 'loser']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

awfully
Probability in positive review: 1.51001e-05
Probability in negative review: 3.01707e-05

slow
Probability in positive review: 0.000119291
Probability in negative review: 0.0001173304

pestering
Probability in positive review: 3.02e-06
Probability in negative review: 3.3523e-06

loser
Probability in positive review: 2.11402e-05
Probability in negative review: 4.02276e-05



In [14]:
# Example 7 candidate words
candidates = ['talent', 'notable', 'freedom', 'attracts']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

talent
Probability in positive review: 0.0001271775
Probability in negarive review: 0.0002191076

notable
Probability in positive review: 3.75011e-05
Probability in negarive review: 3.10403e-05

freedom
Probability in positive review: 7.66326e-05
Probability in negarive review: 1.8259e-05

attracts
Probability in positive review: 8.1524e-06
Probability in negarive review: 5.4777e-06



In [38]:
# Example 8 candidate words
candidates = ['abruptly', 'worst', 'pointless', 'wreck']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

abruptly
Probability in positive review: 6.5219e-06
Probability in negative review: 1.46072e-05

worst
Probability in positive review: 7.17412e-05
Probability in negative review: 0.0004181304

pointless
Probability in positive review: 2.60877e-05
Probability in negative review: 0.0001259869

wreck
Probability in positive review: 2.28267e-05
Probability in negative review: 2.73885e-05



In [43]:
# Example 9 candidates
candidates = ['watchable', 'holes', 'extraordinary', 'miracle']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

watchable
Probability in positive review: 9.7829e-06
Probability in negative review: 3.65179e-05

holes
Probability in positive review: 3.26096e-05
Probability in negative review: 5.84287e-05

extraordinary
Probability in positive review: 5.70668e-05
Probability in negative review: 1.64331e-05

miracle
Probability in positive review: 1.14134e-05
Probability in negative review: 5.4777e-06



In [50]:
# Example 10 candidates
candidates = ['tarantino', 'laugh', 'funny', 'great']
for word in candidates:
    prob_pos, prob_neg = check_prob(word)
    print(word+'\nProbability in positive review: {}\nProbability in negative review: {}\n'.format(
        prob_pos, prob_neg))

tarantino
Probability in positive review: 0.0001190251
Probability in negative review: 2.00849e-05

laugh
Probability in positive review: 0.0001483738
Probability in negative review: 0.0002118041

funny
Probability in positive review: 0.0005592549
Probability in negative review: 0.0007212293

great
Probability in positive review: 0.0010956831
Probability in negative review: 0.0006244568

